In [ ]:
import sys
sys.path.append("..")

import autokoopman.benchmark.fhn as pfhn
import autokoopman as ak
import sympy as sp

from itertools import product
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def lie_derivative(f, F, variables):
    return sp.expand(sum(sp.diff(f, xi)*F[i] for i, xi in enumerate(variables)))
    
def update_basis(F, basis, variables):
    # generate the candidate function
    basis_coef = sp.symbols(" ".join([f"c{i}" for i in range(0, len(basis))]))
    f = sum(ci*bi for ci, bi in zip(basis_coef, basis))

    # lie derivative
    lf = lie_derivative(f, F, variables)

    # remove terms that do not belong to span(*F)
    subs = []
    for term in lf.args:
        in_span = False
        for bf in basis:
            if not any([(term / bf).has(xi) for xi in variables]):
                in_span = True
        if not in_span:
            subs.append([term.has(ci) for ci in basis_coef].index(True))
    return [bi for i, bi in enumerate(basis) if i not in subs]


def obs(F, basis, variables):
    return [lie_derivative(bf, F, variables) for bf in basis]


def generate_monomials(variables, order):
    monomials = set()
    # Generate all combinations of powers including and up to the order for each variable
    for powers in product(range(order + 1), repeat=len(variables)):
        if sum(powers) <= order:
            monomial = prod([var**power for var, power in zip(variables, powers)])
            monomials.add(monomial)
    return list(monomials)

In [ ]:
fhn = pfhn.FitzHughNagumo()
variables = fhn._variables[1:]
exprs = fhn._exprs
x0, x1 = variables

basis = generate_monomials(variables, 3)
#basis = update_basis(exprs, basis, variables)
obs(exprs, basis, variables)[1:]
variables

In [ ]:
lie_obs = ak.core.observables.SymbolicObservable(variables, [x0, x1] + obs(exprs, basis, variables)[1:])

In [ ]:
training_data = fhn.solve_ivps(
    initial_states=np.random.uniform(low=-2.0, high=2.0, size=(30, 2)),
    tspan=[0.0, 1.0],
    sampling_period=0.1
)

# learn model from data
experiment_results = ak.auto_koopman(
    training_data,          # list of trajectories
    sampling_period=0.1,    # sampling period of trajectory snapshots
    obs_type=lie_obs,         # use Random Fourier Features Observables
    opt="monte-carlo",             # grid search to find best hyperparameters
    n_obs=200,              # maximum number of observables to try
    max_opt_iter=200,       # maximum number of optimization iterations
    grid_param_slices=5,   # for grid search, number of slices for each parameter
    n_splits=5,             # k-folds validation for tuning, helps stabilize the scoring
    normalize=False,
    rank=(1, 200, 40)       # rank range (start, stop, step) DMD hyperparameter
)

In [ ]:
# get the model from the experiment results
model = experiment_results['tuned_model']

testing_data = fhn.solve_ivps(
    initial_states=np.random.uniform(low=-2.0, high=2.0, size=(30, 2)),
    tspan=[0.0, 1.0],
    sampling_period=0.1
)

# simulate using the learned model
iv = [0.5, 0.1]
prediction_data = model.solve_ivps(
    initial_states=[t.states[0] for t in testing_data],
    tspan=(0.0, 1.0),
    sampling_period=0.1
)



# plot the results
for trajectory in prediction_data:
    plt.plot(*trajectory.states.T, 'r')
for true_trajectory in testing_data:
    plt.plot(*true_trajectory.states.T, 'k')

In [ ]:
MAX_ITER = 10

basis = generate_monomials(variables, 3) + [sp.exp(-x0**2)*sp.exp(-x1), sp.exp(-x1**2)]
print(basis)
old_len = len(basis)
for _ in range(MAX_ITER):
    print(_)
    basis = update_basis(exprs, basis, variables)
    if len(basis) == old_len:
        break
    old_len = len(basis)
    print(basis)

In [ ]:
variables = sp.symbols('x y')
x, y = variables
F = [x*y+2*x, -1/2*y**2+7*y+1] 
basis = [1, x, y, x*y, x**2, y**2, x**2*y, x*y**2]

MAX_ITER = 10

basis = generate_monomials(variables, 3)
old_len = len(basis)
for _ in range(MAX_ITER):
    print(_)
    basis = update_basis(F, basis, variables)
    if len(basis) == old_len:
        break
    old_len = len(basis)
    print(basis)

In [ ]:
lf = lie_derivative(f, F, variables)

In [ ]:
subs = []
for term in lf.args:
    in_span = False
    for bf in basis:
        if not (term / bf).has(x) and not (term / bf).has(y):
            in_span = True
    if not in_span:
        subs.append([term.has(ci) for ci in basis_coef].index(True))


In [ ]:
basis = [bi for i, bi in enumerate(basis) if i not in subs]

In [ ]:
basis_coef = sp.symbols(" ".join([f"c{i}" for i in range(0, len(basis))]))
alpha_coef = sp.symbols(" ".join([f"a{i}" for i in range(0, len(basis))]))
f = sum(ci*bi for ci, bi in zip(basis_coef, basis))

In [ ]:
lf = lie_derivative(f, F, variables)

In [ ]:
subs = []
for term in lf.args:
    in_span = False
    for bf in basis:
        if not (term / bf).has(x) and not (term / bf).has(y):
            in_span = True
    if not in_span:
        subs.append([term.has(ci) for ci in basis_coef].index(True))

In [ ]:
basis = [bi for i, bi in enumerate(basis) if i not in subs]

In [ ]:
basis